In [ ]:
%matplotlib inline
import string
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import make_interp_spline

**Database connection:**

In [ ]:
database_path = "../build/data/validatorDb/validatorDb.sqlite3"
connection = sqlite3.connect(database_path)

**Rewards and penalties components:**

In [ ]:
SOURCE = "source"
TARGET = "target"
HEAD = "head"
INCLUSION_DELAY = "inclusion_delay"
SYNC_COMMITTEE = "sync_committee"

**Helper functions:**

In [ ]:
def valid_public_key(public_key):
    """Checks whether a string is a valid hex representation of a public key of an Eth2 validator."""
    if len(public_key) != 96:
        return False
    return all(c in string.hexdigits for c in public_key)

def idx(public_key):
    """Returns a validator index corresponding to a public key."""
    
    if public_key.startswith("0x"):
        public_key = public_key[2:]
    
    if not valid_public_key(public_key):
        raise ValueError(f"The string '{public_key}' is not a valid public key of a validator.")
        
    QUERY_FIELD = "validator_index"
    query = f"SELECT {QUERY_FIELD} FROM validators_raw WHERE pubkey=x'{public_key}';"
    query_result = pd.read_sql_query(query, connection)
    
    if len(query_result[QUERY_FIELD]) == 0:
        raise ValueError(f"Not found a validator with a public key '{public_key}'.")
    
    if len(query_result[QUERY_FIELD]) > 1:
        raise ValueError(f"Unexpected result. The 'pubkey' column should be unique. Multiple rows exist for '{public_key}'.")
    
    return query_result[QUERY_FIELD][0]

def get_outcome_var(component):
    return component + "_outcome"

def get_max_reward_var(component):
    return "max_" + component + "_reward"

**Input parameters:**

In [ ]:
start_epoch = 1
end_epoch = 15000
resolution = 1000
rewards = [SOURCE, TARGET, HEAD, INCLUSION_DELAY, SYNC_COMMITTEE]
validators_sets = {
    "set1": list(range(10)),
    "set2": map(idx, [
        "0x8efba2238a00d678306c6258105b058e3c8b0c1f36e821de42da7319c4221b77aa74135dab1860235e19d6515575c381",
        "0xa2dce641f347a9e46f58458390e168fa4b3a0166d74fc495457cb00c8e4054b5d284c62aa0d9578af1996c2e08e36fb6",
        "0x98b7d0eac7ab95d34dbf2b7baa39a8ec451671328c063ab1207c2055d9d5d6f1115403dc5ea19a1111a404823bd9a6e9",
        "0xb0fd08e2e06d1f4d90d0d6843feb543ebeca684cde397fe230e6cdf6f255d234f2c26f4b36c07170dfdfcbbe355d0848",
        "0xab7a5aa955382906be3d76e322343bd439e8690f286ecf2f2a7646363b249f5c133d0501d766ccf1aa1640f0283047b3",
        "0x980c0c001645a00b71c720935ce193e1ed0e917782c4cb07dd476a4fdb7decb8d91daf2770eb413055f0c1d14b5ed6df",
        "0xac7cbdc535ce8254eb9cdedf10d5b1e75de4cd5e91756c3467d0492b01b70b5c6a81530e9849c6b696c8bc157861d0c3",
        "0x98ea289db7ea9714699ec93701a3b6db43900e04ae5497be01fa8cc5a56754c23589eaf1f674de718e291376f452d68c",
        "0x92451d4c099e51f54ab20f5c1a4edf405595c60122ccfb0f39250b7e80986fe0fe457bacd8a887e9087cd6fc323f492c",
        "0xa06f6c678f0129aec056df309a4fe18760116ecaea2292947c5a9cc997632ff437195309783c269ffca7bb2704e675a0"
        ]),
    "set3": [20, 23, 27, 28, 30]
    }

**Query the data from the database:**

In [ ]:
def prepare_query(validators):
    result = "SELECT epoch, validator_index"
    for reward in rewards:
        result += ", " + get_outcome_var(reward) + ", " + get_max_reward_var(reward)
    indices_list = "(" + ",".join(str(v) for v in validators) + ")"
    result += f"""
FROM validator_epoch_info
WHERE epoch >= {start_epoch} AND epoch <= {end_epoch} AND
      validator_index IN {indices_list};
"""
    return result

epoch_info = {}
for key, value in validators_sets.items():
    query = prepare_query(value)
    epoch_info[key] = pd.read_sql_query(query, connection)
connection.close()

**Losses calculation:**

In [ ]:
def sum_max_values(t):
    return sum(getattr(t, get_max_reward_var(reward)) for reward in rewards)

def sum_actual_values(t):
    return sum(getattr(t, get_outcome_var(reward)) for reward in rewards)

total_losses_per_epoch_point = {}
average_losses_per_epoch_point = {}

for sn, ei in epoch_info.items():
    total_losses_per_epoch_point[sn] = {}
    average_losses_per_epoch_point[sn] = {}
    validators_per_epoch_point = {}
    for t in ei.itertuples():
        epoch_point = t.epoch // resolution
        if not epoch_point in total_losses_per_epoch_point[sn]:
            total_losses_per_epoch_point[sn][epoch_point] = 0
            validators_per_epoch_point[epoch_point] = 0
        total_losses_per_epoch_point[sn][epoch_point] += sum_max_values(t) - sum_actual_values(t)
        validators_per_epoch_point[epoch_point] += 1
    for epoch_point, value in total_losses_per_epoch_point[sn].items():
        average_losses_per_epoch_point[sn][epoch_point] = value / validators_per_epoch_point[epoch_point]

**Average losses graph:** 

In [ ]:
plt.subplots(figsize = (20, 5))
plt.title("Average losses per epoch")
plt.xlabel("Epoch")
plt.ylabel("Gwei")

for name, value in average_losses_per_epoch_point.items():
    epochs = np.array([ep * resolution + resolution / 2 for ep in value.keys()])
    values = np.array(list(value.values()))
    spline = make_interp_spline(epochs, values)
    x = np.linspace(epochs.min(), epochs.max(), resolution)
    y = spline(x)
    plt.plot(x, y, label=name)

plt.legend(loc="best")

**Total losses:**

In [ ]:
sets_total_losses = {}
for set_name, epoch_points in total_losses_per_epoch_point.items():
    sets_total_losses[set_name] = 0
    for _, losses in epoch_points.items():
        sets_total_losses[set_name] += losses

plt.title("Total losses")
plt.xlabel("Set")
plt.ylabel("Ethers")
plt.bar(list(sets_total_losses.keys()), [loss * 1e-9 for loss in sets_total_losses.values()])
print(sets_total_losses)